# ROMS netcdf check for initial condition
**Author: Jun Sasaki, Coded on August 30, 2020, Updated on September 1, 2020**<br>
A sample code for manipulating `Projects/Sandy/Sandy_ini.nc`

- Checking an initial condition netcdf file for ROMS in COAWST based on [ROMS Ocean Model](http://xarray.pydata.org/en/stable/examples/ROMS_ocean_model.html) Example.

In [ ]:
import numpy as np
from netCDF4 import num2date, date2num
import datetime
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import xarray as xr
import hvplot.xarray
import holoviews as hv
from geoviews import opts
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Reading grid and initial condition netcdf files
- Variables are consistent with an example of [roms xarray](http://xarray.pydata.org/en/stable/examples/ROMS_ocean_model.html).
- times contains only one element due to initial condition
- See [Coordinate system of (xi, eta)](https://www.myroms.org/wiki/Fractional_Coordinate_System_(%CE%BE_-_%CE%B7_space))
- Info about S coordinate is given in Sandy_ini.nc.
- Bathymetry in positive (m) `dsg.h(eta_rho:65, xi_rho:87)` (65 rows, 87 columns)
- Salinity `ds.salt(time:1, sc_r:16, erho:65, xrho:87)`
- `ds.salt[0,:,0,50]` = array(34.931698, 34.9347  , ..., 36.40495 , 36.405586)
 - Seems to be assigned from surface to bottom but according to HYCOM, surface layer salinity is higher than deep layer salinity (surprisingly); thus assigned from bottom to surface.
- Temperature `ds.temp(time:1, sc_r:16, erho:65, xrho:87)`
- `ds.temp[0,:,0,50]` = array(1.946742,  2.38324 , ..., 26.159126, 26.154554)
 - `ds.temp(sc_r)` is assigned from bottom to surface. 
- S coordinate `ds.sc_r` = array(-0.96875, -0.90625, ..., -0.03125)

In [ ]:
grid_nc = './Sandy_roms_grid.nc'
ini_nc = "./Sandy_ini.nc"
#grid_nc = './training/Sandy_roms_grid.nc'
#ini_nc = "./training/coawst_ini.nc"

with xr.open_dataset(grid_nc) as dsg:
    pass
with xr.open_dataset(ini_nc) as ds:
    pass

In [ ]:
ds.sc_r
ds.temp[0,:,0,50]
ds.salt[0,:,0,50]
ds.zeta
ds

In [ ]:
### Define h in ds using dsg.h
ds['h']=(('erho', 'xrho'), dsg.h)
ds['h'].attrs["long_name"] = "Final bathymetry at RHO-points"
ds['h'].attrs["units"] = "meter"
ds['h'].attrs["field"] = "bath, scalar"

### Set dimension time as coords using ocean_time
### Coords should be consistent with dimension; thus time should be defined.
### Modified Julian Date：MJD
mjd = '1858-11-17 00:00:00'
ds.coords['time'] = (('time'), np.datetime64(mjd) + ds['ocean_time'])
#ds.coords['time'] = ds['ocean_time']
ds['time'].attrs["long_name"] = "Time"
ds.coords['xrho'] = ds['xrho']
ds.coords['erho'] = ds['erho']

## Add a lazily (incrementally 追加的に) calculated vertial coordinates
- This is given in a [roms xarray example](http://xarray.pydata.org/en/stable/examples/ROMS_ocean_model.html).

In [ ]:
if ds.Vtransform == 1:
    Zo_rho = ds.hc * (ds.sc_r - ds.Cs_r) + ds.Cs_r * ds.h
    z_rho = Zo_rho + ds.zeta * (1 + Zo_rho/ds.h)
elif ds.Vtransform == 2:
    Zo_rho = (ds.hc * ds.sc_r + ds.Cs_r * ds.h) / (ds.hc + ds.h)
    z_rho = ds.zeta + (ds.zeta + ds.h) * Zo_rho

ds.coords['z_rho'] = z_rho.transpose()   # needing transpose seems to be an xarray bug

ds['z_rho'].attrs['long_name'] = 'Depth'
ds['z_rho'].attrs['units'] = 'm'
ds['z_rho'].attrs['field'] = 'depth, scalar, series'
ds

In [ ]:
#ds['z_rho'] = ds['z_rho'][::-1,:,:,:]
#ds.coords['z_rho'] = ds['z_rho']
#ds.coords['z_rho']
#print(ds['z_rho'][::-1,62,:,:])
#print(ds['sc_r'])
#ds['zrho'] = ds['z_rho'][::-1,:,:,:]
#ds['zrho'][::-1,62,0,0]
#ds.coords['z_rho']=ds['z_rho'][::-1,:,:,:]
#ds['z_rho'][0,0,0,:]

In [ ]:
z='salt'
cmap='magma_r'
frame_height=300
frame_width=300
project = ccrs.PlateCarree()

# Interactive plotting with [hvPlot](https://hvplot.holoviz.org/)

## Plan view
- Set x-axis range: `xarray.Dataset.hvplot.quadmesh(xlim=(xmin, xmax))`
- Set num of color levels: `p.opts(opts.QuadMesh(color_levels=20))`
- Set value range: `xarray.Dataset.hvplot.quadmesh(clim=(cmin, cmax))`

In [ ]:
line_alpha = None  ###
clim = None  ### (cmin, cmax) or None
ds.hvplot.quadmesh(groupby=['sc_r','time'], x='xrho', y='erho',z=z , \
                   geo=True , project=project, tiles=None, coastline='10m', \
                   frame_height=frame_height, clim=clim, \
                   cmap=cmap, line_color='aqua', line_alpha=line_alpha, line_width=0.1).opts(
                   opts.QuadMesh(color_levels=20))

### Gets available options for hvplot.quadmesh()
- Plot options are specified by `quadmesh().opts()`

In [ ]:
#hv.help(hv.QuadMesh)

## Vertical sectional views in S coordinate

In [ ]:
ds.hvplot.quadmesh(groupby=['xrho','time'], x='erho', y='sc_r',z=z , \
                   frame_height=frame_height, frame_width=frame_width, cmap=cmap, \
                   line_color='aqua', line_alpha=1, line_width=0.1)

In [ ]:
ds.hvplot.quadmesh(groupby=['erho','time'], x='xrho', y='sc_r',z=z , \
                   frame_height=frame_height, frame_width=frame_width, cmap=cmap, \
                   line_color='aqua', line_alpha=1, line_width=0.1)

## Vertical sectional views in z coordinate

In [ ]:
ds.hvplot.quadmesh(groupby=['xrho','time'], x='erho', y='z_rho',z=z , \
                   frame_height=frame_height, frame_width=frame_width, cmap=cmap, \
                   line_color='aqua', line_alpha=1, line_width=0.1)

In [ ]:
ds.hvplot.quadmesh(groupby=['erho','time'], x='xrho', y='z_rho',z=z , \
                   frame_height=frame_height, frame_width=frame_width, cmap=cmap, \
                   line_color='aqua', line_alpha=1, line_width=0.1).opts(bgcolor='lightgray')

# Plotting with matplotlib

## Plan view

In [ ]:
ds[z].isel(sc_r=0, time=0).plot(x='xrho', y='erho', extend='both', cmap='magma_r')

In [ ]:
p=ds[z].isel(sc_r=0, time=0).plot(x='xrho', y='erho', extend='both', cmap='magma_r',\
                                           vmin=31, vmax=36, alpha=1, edgecolor='aqua', linewidth=0.01)
p.axes.set_xlim(40, 70)
p.axes.set_ylim(30, 50)
p.figure.savefig('temp.png', dpi=300, bbox_inches='tight')

## Vertical sectional views in S coordinate

In [ ]:
ds[z].isel(erho=0, time=0).plot.pcolormesh(x='xrho', y='sc_r', extend='both', cmap='magma_r')

In [ ]:
ds[z].isel(xrho=0, time=0).plot.pcolormesh(x='erho', y='sc_r', extend='both', cmap='magma_r')

## Vertical sectional view in z coordinate

In [ ]:
ds[z].isel(erho=0, time=0).plot.pcolormesh(x='xrho', y='z_rho', extend='both', cmap='magma_r')

In [ ]:
p=ds[z].isel(xrho=15, time=0).plot.pcolormesh(x='erho', y='z_rho', extend='both', \
                                                        cmap='magma_r')
p.axes.set_facecolor('lightgray')